# Shopping Mall

## 장바구니 분석 파일 생성

---

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

plt.rc('figure', figsize=(10, 6))

from matplotlib import rcParams
rcParams['font.family'] = 'NanumGothic'
rcParams['font.size'] = 10

### 파일 로드

In [ ]:
df = pd.read_excel('data/master.xlsx', sheet_name='order',encoding='cp949')
df

In [ ]:
df.columns

In [ ]:
df1 = df[['주문번호','주문일시','진행구분','배송지','상품번호','sex']].copy()
df1.head()

###  성별 결측치 처리

In [ ]:
df1['sex'].value_counts()

In [ ]:
df1['sex'].isnull().sum()

In [ ]:
df1['sex'].fillna('U',inplace=True)

In [ ]:
df1['sex'].replace('-','U', inplace=True)

# apply 활용
#df1['sex'] = df1['sex'].apply(lambda x: 'U' if x == '-' else x)

# loc 활용
#df1.loc[df1['sex'] == '-', 'sex'] = 'U'

In [ ]:
df1['sex'].value_counts()

### 파일 로드: multi_2_value

In [ ]:
df_m = pd.read_excel('data/master.xlsx', sheet_name='multi_2_value',encoding='cp949')

In [ ]:
# 컬럼 이름 수정
df_m.rename(columns = {1:'item1', 2:'item2', 3:'item3',  4:'item4',  5:'item5',    6:'item6',
                       7:'item7', 8:'item8', 9:'item9', 10:'item10', 11:'item11', 12:'item12'},
            inplace = True)

In [ ]:
# 결측치 제거: 모두 NaN 인 행 제거
df_m.dropna(axis=1, how='all', inplace=True)

In [ ]:
df_m.head()

In [ ]:
len(df_m)

### 주문일시, 성별 추가 : merge( on='주문번호') 이용

In [ ]:
# 중복제거
df2 = df1[['주문번호','주문일시','sex']].drop_duplicates('주문번호')

In [ ]:
df_m1 = pd.merge(df2, df_m, on='주문번호')

In [ ]:
df_m1

### 요일 추가 ( wday )

In [ ]:
# datetime 형식으로 변경
df_m1['주문일시'] = pd.to_datetime(df_m1['주문일시'])

In [ ]:
# 요일 컬럼 추가
df_m1['wday'] = df_m1['주문일시'].dt.dayofweek    # 0: Monday, 1: Tuesday, ...
df_m1
#df_m1['주문일시'].dt.day_name()

### item 컬럼만 추출

In [ ]:
col_list = ['item1','item2','item3','item4', 'item5', 'item6',
            'item7','item8','item9','item10','item11','item12']

In [ ]:
# item 컬럼만 추출(요일별)
df_m1.loc[df_m1['wday'] == 0, col_list]    # Monday

In [ ]:
# item 컬럼만 추출(성별: Female, Male, Unknown)
df_m1.loc[df_m1['sex'] == 'F', col_list]    # Female

### 결과 파일 저장 ( 요일별 파일 생성 )

In [ ]:
# Monday
#df_m1.loc[df_m1['wday'] == 0, col_list].to_csv('data/basket_mon.csv', index=False, na_rep='')

In [ ]:
suffix = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
fname = 'data/result/basket_{}_{}.csv'

for wday in range(7):
    df_tmp = df_m1.loc[df_m1['wday'] == wday, col_list]
    df_tmp.to_csv(fname.format(wday,suffix[wday]), index=False, na_rep='')

### 결과 파일 저장 ( 성별 파일 생성 )

In [ ]:
# Female
#df_m1.loc[df_m1['sex'] == 'F', col_list].to_csv('data/basket_female.csv', index=False, na_rep='')

In [ ]:
suffix = {'F':'Female','M':'Male','U':'Unknown'}
fname = 'data/result/basket_{}.csv'

for key, value in suffix.items():
    df_tmp = df_m1.loc[df_m1['sex'] == key, col_list]
    df_tmp.to_csv(fname.format(value), index=False, na_rep='')

In [ ]:
# end of file